## RAG

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_ollama import OllamaEmbeddings

In [3]:
base_url = "http://localhost:11434"
embedding_model = "nomic-embed-text"

embeddings = OllamaEmbeddings(model=embedding_model, base_url=base_url)

In [4]:
db_path = "./../store/db_supplements"
vector_store = FAISS.load_local(folder_path=db_path,
                                embeddings=embeddings,
                                allow_dangerous_deserialization=True)

In [ ]:
# Test the vector store
question = "Is creatine useful to increase muscle mass?"
relevant_chunks = vector_store.search(query=question,
                                      search_type="similarity",
                                      k=5)
relevant_chunks

[Document(metadata={'source': '.././data/rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'file_path': '.././data/rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'page': 0, 'total_pages': 15, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241021113850Z', 'trapped': ''}, page_content='acids than traditional protein sources. Its numerous benefits have made it a popular choice\nfor snacks and drinks among consumers [3]. Another widely embraced supplement is\ncaffeine, which is found in many sports and food supplements. Caffeine reduces perceived\neffort, minimizes fatigue and pain, and proves to be effective for endurance and high-\nintensity activities, which is the choice of consumers [4].\nCreatine monohydrate is another well-known supplement used to gain muscle mass\nand support performance and recovery. It is known not to 

In [6]:
# Convert vector store to retriever
retriever = vector_store.as_retriever(search_type="similarity",
                                      search_kwargs={'k': 3})
retriever.invoke(question)

[Document(metadata={'source': '.././data/rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'file_path': '.././data/rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'page': 0, 'total_pages': 15, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241021113850Z', 'trapped': ''}, page_content='acids than traditional protein sources. Its numerous benefits have made it a popular choice\nfor snacks and drinks among consumers [3]. Another widely embraced supplement is\ncaffeine, which is found in many sports and food supplements. Caffeine reduces perceived\neffort, minimizes fatigue and pain, and proves to be effective for endurance and high-\nintensity activities, which is the choice of consumers [4].\nCreatine monohydrate is another well-known supplement used to gain muscle mass\nand support performance and recovery. It is known not to 

In [10]:
# Convert vector store to retriever
retriever = vector_store.as_retriever(search_type="similarity_score_threshold",
                                      search_kwargs={'k': 3, 'score_threshold': 0.6})
retriever.invoke(question)

[Document(metadata={'source': '.././data/rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'file_path': '.././data/rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'page': 0, 'total_pages': 15, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241021113850Z', 'trapped': ''}, page_content='acids than traditional protein sources. Its numerous benefits have made it a popular choice\nfor snacks and drinks among consumers [3]. Another widely embraced supplement is\ncaffeine, which is found in many sports and food supplements. Caffeine reduces perceived\neffort, minimizes fatigue and pain, and proves to be effective for endurance and high-\nintensity activities, which is the choice of consumers [4].\nCreatine monohydrate is another well-known supplement used to gain muscle mass\nand support performance and recovery. It is known not to 

In [11]:
# Convert vector store to retriever
retriever = vector_store.as_retriever(search_type="mmr",
                                      search_kwargs={'k': 3, 'fetch_k': 10, 'lambda_multi': 0.3})
retriever.invoke(question)

[Document(metadata={'source': '.././data/rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'file_path': '.././data/rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'page': 0, 'total_pages': 15, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241021113850Z', 'trapped': ''}, page_content='acids than traditional protein sources. Its numerous benefits have made it a popular choice\nfor snacks and drinks among consumers [3]. Another widely embraced supplement is\ncaffeine, which is found in many sports and food supplements. Caffeine reduces perceived\neffort, minimizes fatigue and pain, and proves to be effective for endurance and high-\nintensity activities, which is the choice of consumers [4].\nCreatine monohydrate is another well-known supplement used to gain muscle mass\nand support performance and recovery. It is known not to 

### RAG with LLAMA

In [12]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub

In [ ]:
# We can use prompt from langchain hub
prompt = hub.pull("rlm/rag-prompt")
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [19]:
# Or we can just create our own so it's more flexible
prompt = """
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

Question: {question} 
Context: {context} 
Answer:
"""

template = ChatPromptTemplate.from_template(prompt)
template.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\nYou are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\n\nQuestion: {question} \nContext: {context} \nAnswer:\n"), additional_kwargs={})]

In [20]:
# Llama model
model_name = "llama3.2:1b"
base_url = "http://localhost:11434"
llm = ChatOllama(model=model_name, base_url=base_url)

In [21]:
def combine_docs(docs):
  """Function to combine retrieved docs to create a context."""
  return "\n\n".join([doc.page_content for doc in docs])

In [22]:
rag_chain = (
  {"context": retriever | combine_docs, "question": RunnablePassthrough()}
  | template
  | llm 
  | StrOutputParser()
)

In [24]:
question = "Is creatine useful to increase muscle mass?"
response = rag_chain.invoke(question)
print(response)

Here's the answer:

Creatine monohydrate is effective for muscle mass gain and can be used to support performance and recovery, although its benefits may vary depending on individual factors such as training frequency and supplement usage.

I don't know if creatine increases fat mass or remains effective even when taken in recommended doses.


In [27]:
# Try different prompt
prompt = """
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in bullet points. 

Question: {question} 
Context: {context} 
Answer:
"""

template = ChatPromptTemplate.from_template(prompt)

rag_chain = (
  {"context": retriever | combine_docs, "question": RunnablePassthrough()}
  | template
  | llm 
  | StrOutputParser()
)

question = "Is creatine useful to increase muscle mass?"
response = rag_chain.invoke(question)
print(response)

Here are the answers to the question in bullet points:

• Creatine monohydrate can be useful for increasing muscle mass.
• Caffeine also has benefits when taken as a supplement and supports endurance and high-intensity activities.
• However, creatine monohydrate is known not to increase fat mass and remains effective even at recommended doses.
• Protein supplementation is more common among younger fitness center users compared to older ones, while vitamin supplementation shows similar trends.
• The correlation between training frequency and supplement usage is weak, but there is evidence that individuals who take higher amounts of supplements are less active.
• There is no clear evidence on the safety or effectiveness of weight-loss sports and bodybuilding supplements when taken as intended, with potential risks associated.
